## Análise explanatória Empresas Ativas (EMPRESAS)

#### Realização de uma análise explanatória detalhada dos dados das empresas ativas da cidade do Recife, verificando possíveis inconsistencias ou duplicações nos registros.

##### 🟩🟩 Importando Bibliotecas

In [20]:
import pandas as pd
import numpy as np
import os

In [21]:
# Diretório onde os arquivos CSV estão localizados
CSV_FOLDER = r'./datasets/CSV_Separados'

# Nome do arquivo CSV que você quer carregar
CSV_FILE = 'EMPRESAS.csv'  # Substitua pelo nome do arquivo gerado

# Caminho completo do arquivo CSV
csv_path = os.path.join(CSV_FOLDER, CSV_FILE)

# Verificar se o arquivo existe antes de tentar carregá-lo
if os.path.exists(csv_path):
    # Carregar o arquivo CSV como DataFrame
    df = pd.read_csv(csv_path)
    print(f"Arquivo CSV '{CSV_FILE}' carregado com sucesso!")
else:
    print(f"Erro: O arquivo '{csv_path}' não foi encontrado.")

pd.set_option('display.max_columns', None)  # Exibe todas as colunas
pd.set_option('display.max_colwidth', None)  # Exibe o conteúdo completo de cada célula
pd.set_option('display.width', None)  # Ajuste de largura para evitar quebra de linha

Arquivo CSV 'EMPRESAS.csv' carregado com sucesso!


In [22]:
df.head(5)

,CNPJ BASICO,RAZAO SOCIAL / NOME EMPRESARIAL,NATUREZA JURIDICA,QUALIFICACAO DO SOCIO,CAPITAL SOCIAL DA EMPRESA,PORTE DA EMPRESA,ENTE FEDERATIVO RESPONSAVEL
0,0,BANCO DO BRASIL SA,SOCIEDADE DE ECONOMIA MISTA,DIRETOR,"120000000000,00",DEMAIS,NaN
1,633,INSTITUTO QUALIDADE NO ENSINO,ASSOCIAÇÃO PRIVADA,PRESIDENTE,"0,00",DEMAIS,NaN
2,802,HC PNEUS S/A,SOCIEDADE ANÔNIMA FECHADA,DIRETOR,"204418998,00",DEMAIS,NaN
3,2927,TRANSMETA TRANSPORTES RODOVIARIOS DE CARGAS LTDA,SOCIEDADE EMPRESÁRIA LIMITADA,SÓCIO-ADMINISTRADOR,"800000,00",DEMAIS,NaN
4,5050,INTEGRE DO BRASIL S/A INDUSTRIA COM IMP EXP TRANSPORTES,SOCIEDADE ANÔNIMA FECHADA,PRESIDENTE,"0,00",DEMAIS,NaN


##### 🟡 ENTE FEDERATIVO RESPONSÁVEL tem nulls ...vamos olhar melhor

In [23]:
valores_unicos = df['ENTE FEDERATIVO RESPONSAVEL'].unique()
print(valores_unicos)

[nan 'UNIÃO' 'PERNAMBUCO' 'RECIFE - PE' 'PIAUI' 'MARAIAL - PE'
 'RIO GRANDE DO SUL' 'ALAGOAS']


##### 🟩🟩 Verificando a quantidade de nulos por coluna (mostrando apenas as colunas com nulos)


In [24]:
# Contar os valores nulos por coluna
nulos_por_coluna = df.isna().sum()

# Filtrar as colunas com valores nulos (maior que 0)
colunas_com_nulos = nulos_por_coluna[nulos_por_coluna > 0]

# Exibir as colunas com valores nulos
print(colunas_com_nulos)



QUALIFICACAO DO SOCIO              33
ENTE FEDERATIVO RESPONSAVEL    532238
dtype: int64


##### 🟩🟩 Verificando dados duplicados 

In [25]:
duplicatas = df.duplicated()

# Exibir as linhas duplicadas (se houver)
print(df[duplicatas])

Empty DataFrame
Columns: [CNPJ BASICO, RAZAO SOCIAL / NOME EMPRESARIAL, NATUREZA JURIDICA, QUALIFICACAO DO SOCIO, CAPITAL SOCIAL DA EMPRESA, PORTE DA EMPRESA, ENTE FEDERATIVO RESPONSAVEL]
Index: []


##### Não há dados duplicados na tabela empresas


##### 🟩🟩 Verificando os tipos dos dados 

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532580 entries, 0 to 532579
Data columns (total 7 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   CNPJ BASICO                      532580 non-null  int64 
 1   RAZAO SOCIAL / NOME EMPRESARIAL  532580 non-null  object
 2   NATUREZA JURIDICA                532580 non-null  object
 3   QUALIFICACAO DO SOCIO            532547 non-null  object
 4   CAPITAL SOCIAL DA EMPRESA        532580 non-null  object
 5   PORTE DA EMPRESA                 532580 non-null  object
 6   ENTE FEDERATIVO RESPONSAVEL      342 non-null     object
dtypes: int64(1), object(6)
memory usage: 28.4+ MB


##### 🟡 Em uma analise por data type é preciso ajustar 'CNPJ BASICO' para object pois ele não possui a função de soma visto que se trata de códigos

##### 🟩🟩 Alterando Dtype para object 

In [27]:
conversao = ['CNPJ BASICO']

# Converter as colunas específicas de 'int' para 'object'
df[conversao] = df[conversao].astype('object')

# Verificar os tipos de dados após a conversão
df.dtypes

CNPJ BASICO                        object
RAZAO SOCIAL / NOME EMPRESARIAL    object
NATUREZA JURIDICA                  object
QUALIFICACAO DO SOCIO              object
CAPITAL SOCIAL DA EMPRESA          object
PORTE DA EMPRESA                   object
ENTE FEDERATIVO RESPONSAVEL        object
dtype: object

In [28]:
df['CNPJ BASICO'] = df['CNPJ BASICO'].astype(str)

# Adicionar zeros à esquerda para garantir 8 caracteres
df['CNPJ BASICO'] = df['CNPJ BASICO'].str.zfill(8)

In [29]:
df['CNPJ BASICO']

0         00000000
1         00000633
2         00000802
3         00002927
4         00005050
            ...   
532575    13845294
532576    13845315
532577    13845552
532578    13845851
532579    13845923
Name: CNPJ BASICO, Length: 532580, dtype: object

#### Analises

* QUANTIDADE DE QUALIFICAÇÃO DE SOCIO
* QUANTIDADE DE SOCIO POR CNPJ
* VALORES DO CAPITAL DO MAIOR AO MENOR 

In [30]:
# 1. QUANTIDADE DE QUALIFICAÇÃO DE SÓCIO POR CNPJ
qualificacoes_unicas = (
    df.groupby("CNPJ BASICO")["QUALIFICACAO DO SOCIO"]
    .nunique()
    .reset_index(name="QUANTIDADE_QUALIFICACOES")
)

# Exibir os resultados
print("1. Quantidade de Qualificação de Sócio:")
qualificacoes_unicas


1. Quantidade de Qualificação de Sócio:


,CNPJ BASICO,QUANTIDADE_QUALIFICACOES
0,00000000,1
1,00000633,1
2,00000802,1
3,00002927,1
4,00005050,1
...,...,...
532575,97755177,1
532576,98311715,1
532577,98406010,1
532578,98408073,1


In [31]:
# 2. QUANTIDADE DE SÓCIO POR CNPJ
quantidade_socios = (
    df.groupby("CNPJ BASICO")
    .size()
    .reset_index(name="QUANTIDADE_SOCIOS")
)

print("\n2. Quantidade de Sócio por CNPJ:")
quantidade_socios




2. Quantidade de Sócio por CNPJ:


,CNPJ BASICO,QUANTIDADE_SOCIOS
0,00000000,1
1,00000633,1
2,00000802,1
3,00002927,1
4,00005050,1
...,...,...
532575,97755177,1
532576,98311715,1
532577,98406010,1
532578,98408073,1


In [32]:
# Obter os valores únicos da coluna "QUANTIDADE_SOCIOS"
valores_unicos = quantidade_socios["QUANTIDADE_SOCIOS"].unique()

# Exibir os valores únicos
print("Valores únicos na coluna 'QUANTIDADE_SOCIOS':")
print(valores_unicos)


Valores únicos na coluna 'QUANTIDADE_SOCIOS':
[1]


In [33]:
ocorrencias = df["QUALIFICACAO DO SOCIO"].value_counts()
ocorrencias

QUALIFICACAO DO SOCIO
EMPRESÁRIO                                                  325918
SÓCIO-ADMINISTRADOR                                         157008
PRESIDENTE                                                   13179
CANDIDATO A CARGO POLÍTICO ELETIVO                            8139
TITULAR DE EMPRESA INDIVIDUAL IMOBILIÁRIA                     7592
ADMINISTRADOR                                                 6937
TITULAR PESSOA FÍSICA RESIDENTE OU DOMICILIADO NO BRASIL      5111
SÍNDICO (CONDOMÍNIO)                                          4741
DIRETOR                                                       1991
SÓCIO-GERENTE                                                  850
ADMINISTRADOR JUDICIAL                                         618
LIQUIDANTE                                                     117
INVENTARIANTE                                                  109
NÃO INFORMADA                                                   39
TABELIÃO                                

In [34]:
razao_social_por_cnpj = df.groupby("CNPJ BASICO")["RAZAO SOCIAL / NOME EMPRESARIAL"].nunique().reset_index(name="QUANTIDADE_RAZOES_SOCIAIS")
razao_social_por_cnpj

,CNPJ BASICO,QUANTIDADE_RAZOES_SOCIAIS
0,00000000,1
1,00000633,1
2,00000802,1
3,00002927,1
4,00005050,1
...,...,...
532575,97755177,1
532576,98311715,1
532577,98406010,1
532578,98408073,1


In [35]:

# 3. VALORES DO CAPITAL SOCIAL (MAIOR PARA MENOR)
capital_ordenado = (
    df[["CNPJ BASICO", "CAPITAL SOCIAL DA EMPRESA"]]
    .drop_duplicates()
    .sort_values(by="CAPITAL SOCIAL DA EMPRESA", ascending=False)
)


print("\n3. Valores do Capital Social (Maior para Menor):")
capital_ordenado


3. Valores do Capital Social (Maior para Menor):


,CNPJ BASICO,CAPITAL SOCIAL DA EMPRESA
275484,57082199,"9999999,99"
380184,37101549,"99999,99"
244239,52452731,"99999,99"
265322,55277002,"99999,99"
238232,51682216,"99999,99"
...,...,...
72661,08138786,"0,00"
369603,35717875,"0,00"
369602,35717867,"0,00"
32897,03452134,"0,00"


In [36]:
resultado = df[df["CNPJ BASICO"] == '57082199']
resultado

,CNPJ BASICO,RAZAO SOCIAL / NOME EMPRESARIAL,NATUREZA JURIDICA,QUALIFICACAO DO SOCIO,CAPITAL SOCIAL DA EMPRESA,PORTE DA EMPRESA,ENTE FEDERATIVO RESPONSAVEL
275484,57082199,57.082.199 SARAH MAYARA DE SOUZA MARTINS,EMPRESÁRIO (INDIVIDUAL),EMPRESÁRIO,"9999999,99",MICRO EMPRESA,NaN


##### 🟩🟩 Salvando o dataframe EMPRESAS

In [37]:
# Pasta para salvar o arquivo
OUTPUT_FOLDER = r'C:\Users\carli\Desktop\EmpresasAtivas\datasets\CSV_Tratados'
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [38]:
# Salva o DataFrame em formato CSV com separador e controle de casas decimais
df.to_csv('C:\\Users\\carli\\Desktop\\EmpresasAtivas\\datasets\\CSV_Tratados\\empresas.csv', sep=';', decimal=',', encoding='UTF-8', index=False)
print('Salvo com sucesso')

Salvo com sucesso
